# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:45<00:00,  9.16s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Satellai Smart Collar and GPS Dog Fence for $425 + free shipping\nDetails: It's $75 off. Customers ordered the product during the activity period can enjoy 15% off early bird discount and get 2-months free subscription. Buy Now at SATELLAI\nFeatures: dual antennas and dual-frequency no terrain limits real-time tracking 120-hour battery\nURL: https://www.dealnews.com/Satellai-Smart-Collar-and-GPS-Dog-Fence-for-425-free-shipping/21731995.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: GoPro Hero11 Black Mini Action Camera for $130 for members + free shipping
Details: My Best Buy Total members can get this model for the best-ever price we've seen. (It's also a low now by $70). Best Buy Total are paid memberships, which cost $50 and $180 annually respectively. Buy Now at Best Buy
Features: cinematic 5.3K60 + 2.7K240 video 24.7MP stills from video d

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [14]:
result.deals[1]

Deal(product_description='The INIU 45W 10,000mAh Slim Power Bank provides a compact solution for charging your devices on the go. This portable charger comes with a built-in USB-C cable, making it easy to connect without additional accessories. With 10,000mAh capacity, it can charge most smartphones multiple times, ensuring you stay powered up throughout the day. The sleek and slim design allows for easy transport, fitting comfortably in pockets or bags.', price=14.0, url='https://www.dealnews.com/products/INIU/INIU-45-W-10-000-m-Ah-Slim-Power-Bank/488639.html?iref=rss-c142')

In [15]:
from agents.scanner_agent import ScannerAgent

In [16]:
agent = ScannerAgent()
result = agent.scan()

In [17]:
result

DealSelection(deals=[Deal(product_description='The GoPro Hero11 Black Mini Action Camera is an advanced action camera designed for capturing high-quality video and photos. It features cinematic 5.3K video at 60 frames per second, allowing for stunningly smooth footage, as well as the ability to capture 24.7MP still images from video. The camera is also built to endure tough conditions, being waterproof up to 33 feet, and comes with dual mounting fingers for versatile attachment options. Included is a carrying case, making it easy to transport and protect this rugged piece of technology.', price=130.0, url='https://www.dealnews.com/products/Go-Pro/Go-Pro-Hero11-Black-Mini-Action-Camera/396992.html?iref=rss-c142'), Deal(product_description='The Samsung 990 EVO Plus 2TB Internal SSD is a high-performance storage solution designed for gamers and content creators. It offers impressive read and write speeds of up to 7,250MB/s and 6,300MB/s, respectively, significantly improving load times an